In [1]:
"""Build Baseline Location Classifier
Approaches:
- instance representation in form of 'bag' of words 
    - features: word frequencies, metadata 
    - exclude rare words in data set (used by less than 3 users )
- gramatical structure with NLP
- model instances in terms of authors instead of documents 


- Baseline Classifier: Naive Bayes Model ()

"""


"Build Baseline Location Classifier\nApproaches:\n- instance representation in form of 'bag' of words \n    - features: word frequencies, metadata \n    - exclude rare words in data set (used by less than 3 users )\n- gramatical structure with NLP\n- model instances in terms of authors instead of documents \n\n\n- Baseline Classifier: Naive Bayes Model ()\n\n"

In [83]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.preprocessing import OneHotEncoder

from os import path, getcwd
from collections import defaultdict
import nltk
import re, string
from validators import url
from info_gain.info_gain import info_gain_ratio

from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

# import preprocessor as p
# nltk.download('english')
# nltk.download('punkt')

In [84]:
"""
Import data into pandas format from original csv/tsv file formats for 
the top 10/50/100 train/test/dev data 
"""

fdir = path.join(getcwd(), "2019S1-proj2-datah")

train_data = "train-top10.csv"
test_data = "test-top10.csv"
dev_data = "dev-top10.csv"

train_fpath = path.join(fdir, train_data)
test_fpath = path.join(fdir, test_data)
dev_fpath = path.join(fdir, dev_data)

train_top10 = pd.read_csv(train_fpath, encoding="utf_8")
test_top10 = pd.read_csv(test_fpath, encoding="utf_8")
dev_top10 = pd.read_csv(dev_fpath, encoding="utf_8")

In [155]:
train = train_top10.iloc[:, 1:35]
test = train_top10.iloc[:, 1:35]
dev = train_top10.iloc[:, 1:35]

train_y = train.iloc[:, -1:]
test_y = test.iloc[:, -1:]
dev_y = dev.iloc[:, -1:]


In [209]:
class GeoTagger:
    _FEATURE_SELECTION = ["info_gain_ratio", "word_locality_heuristic", 'chi_squared', 'mutial_info']
    _VOTING_STRATEGY = ["simple_voting", "bagging", "stacking", "random_forest", "boosting"]
    _CLASSIFIERS = ["Zero-R", "One-R", "Decision-Tree", "MultinomialNB", "SVM", "SemiSupervised", "KNN"]
    _EVALUATION_METRIC = ["Accuracy", "Precision", "Recall"]
#     _EVALUATION METHOD = ["Cross-Validation"]
                          
    def __init__(self, classifier_set, voting_strategy, feature_selection, eval_metric, seed, train_x, train_y, dev_x, dev_y):
        #self.stopwords = set(stopwords.words('english'))
        #self.stemmer = WordNetLemmatizer('english')
    
        self.exclude = set()
        self.classifier_set = classifier_set
        self.voting_strategy = voting_strategy
        self.feature_selection = feature_selection
        self.eval_metric = eval_metric
        np.random.seed(seed)
        self.train_x = train_x
        self.train_y= train_y
        self.dev_x = dev_x
        self.dev_y= dev_y
            
        #train classifier
        print(len(self.classifier_set))
        if len(self.classifier_set)>1: #classifier combination
            clf = self.combine_classifiers(self.train_x, self.train_y, self.classifier_set)
        else: #baseline classifier
            clf = self.train_baseline(self.train_x, self.train_y, self.classifier_set[0])
        
        #predict the class labels of a set of test data
        ybar = self.predict(clf, self.dev_x, self.classifier_set)
        
        #evaluate classifier performance
        score = self.evaluate(ybar, self.dev_y, self.eval_metric)
        
        print("score")
    def train_baseline(self, X, y, classifier):
        """
        trains a single classifier given the training data and their corresponding class labels
        """
        print
        print(GeoTagger._CLASSIFIERS.index(classifier))
        
        if GeoTagger._CLASSIFIERS.index(classifier)==0:
            clf = DummyClassifier(strategy='most_frequent')
            clf.fit(X, y)
        elif GeoTagger._CLASSIFIERS.index(classifier)==1:
            clf = DecisionTreeClassifier(max_depth=1, criterion="entropy")
            clf.fit(X,y)
        elif GeoTagger._CLASSIFIERS.index(classifier)==2:
            clf = DecisionTreeClassifier(max_depth=1, criterion="entropy")
            clf.fit(X,y)
        elif GeoTagger._CLASSIFIERS.index(classifier)==3:
            clf = MultinomialNB()
            clf.fit(X,y)
            print(clf)
        return clf
    
    def combine_classifiers(self, X, y, classifiers):
        pass
    
    def predict(self, clf, X, classifier_type):
        print(clf)
#         print(X)
#         print(classifier_type)
        
        if GeoTagger._CLASSIFIERS.index(classifier_type)==0:
            predictions = clf.predict(X)
        elif GeoTagger._CLASSIFIERS.index(classifier_type)==1:
            predictions = clf.predict(X)
        elif GeoTagger._CLASSIFIERS.index(classifier_type)==2:
            predictions = clf.predict(X)
        elif GeoTagger._CLASSIFIERS.index(classifier_type)==3:
            print("kucing")
            predictions = clf.predict(X)
            
        return predictions
    
    def evaluate(self, ybar, y, metric):
        #TODO: eval method
        
        if GeoTagger._EVALUATION_METRIC.index(metric) == 0:
            score = accuracy_score(ybar, y)
        if GeoTagger._EVALUATION_METRIC.index(metric) == 1:
            score = accuracy_score(ybar, y)
        if GeoTagger._EVALUATION_METRIC.index(metric) == 2:
            score = accuracy_score(ybar, y)
        if GeoTagger._EVALUATION_METRIC.index(metric) == 3:
            score = accuracy_score(ybar, y)
        
        return score
    
    def preprocess(self, x, train=False):
        """
         - Filter rare words (urls, typos rare names, punctuation symbols)
         - calculate word frequencies 
         - metadata
        """
#         if train:
#             feature_weight(x)
#             feature_selection(x)
#       self.class_feature_sets = {label: defaultdict() for label in self.classes}
        x = self.filter(x)
        return x
    
    def filter(self, x):
        """
        https://towardsdatascience.com/extracting-twitter-data-pre-processing-and-sentiment-analysis-using-python-3-0-7192bd8b47cf
        """
        x.loc[x.index.values, 'Text'] = x['Text'].apply(self.filter_tweet)
        return x        
        
    def feature_selection(self, x):
        """
        (1) Information Gain Ratio (IGR) - across all states S, is 
            defined as the ratio between its information gain value IG, 
            which measures the decrease in class entropy H that w brings,
            and its intrinsic entropy IV, which measures the entropy of 
            the presence versus the absence of that word
            
        (2) Word Locality Heuristic (WLH) - promotes words primarily 
            associated with one location. measure the probability of 
            a word occurring in a state, divided by its probability to 
            appear in any state. Then, for a given word w, we define the 
            WLH as the maximum such probability across all the states S
        """
        if self.feature_selection == "IGR":
#             return information_gain_ratio(x)
             return
        elif self.feature_selection == "WLH":
            return word_locality_weight(x)
#         elif seld
        else:
            print("Invalid Feature Selection method: {}. Choose one of \
            ({})".format(self.feature_selection, ", ".join(GeoTagger._FEATURE_SELECTION)))
        
    def word_locality_weight(self, x):
        """
        calculate frequencies of data 
        Measure frequency and divide by sum of freqencies of all words
        """
    
    def information_gain_ratio(self, x):
#         return info_gain_ratio
        pass
        
    def filter_tweet(self, tweet):
        filtered = [self.filter_word(w) for w in tweet.split() if w not in self.stopwords]
        return ' '.join(filtered)
        
    def filter_word(self, word):
        word = word.lower()
        # extract keywords from hashtag 
        if self._is_hyperlink(word):
            print(word)
            return ''
        elif self._is_hashtag(word):
            word = self._process_hashtag(word)
        # potentially cross-reference individuals mentioned? or discard
        elif self._is_mention(word):
            word = self._process_mention(word)
        # remove ascii characters 
        else:
            word = self._ascii_to_unicode(word)
            word = self._word_stem(word)
            # .decode("unicode_escape").encode('utf-8')
            word = re.sub(r'[^\w\s]','', word)
#         print(word)
        return word
                
    def _is_hashtag(self, word):
        if len(word) == 0:
            return False
        return word[0] == "#"
    
    def _is_mention(self, word):
        if len(word) == 0:
            return False
        return word[0] == "@"
    
    def _is_hyperlink(self, word):
        return url(word)
    
    def _process_hashtag(self, word):
        return word[1:]
    
    def _process_mention(self, word):
        return word[1:]
    
    def _ascii_to_unicode(self, word):
        for uescape in re.findall(r'(\\u[0-9a-z]{4})', word):
            try:
#                 print(uescape.encode('utf-8').decode('unicode-escape'), type(uescape.encode('utf-8').decode('unicode-escape')))
#                 word = re.sub(uescape, uescape.encode('utf-8'), word)
#                 print(word)
#                 print(uescape, type(uescape))
                word = re.sub(uescape, ' ', word)  
            except UnicodeDecodeError:
                print("Failed to decode: {}".format(uescape))
        return word
    
    def _word_stem(self, word):
        return self.stemmer.stem(word)
    

In [210]:
gt = GeoTagger(["MultinomialNB"], "simple_voting", "mutual syntax", "Accuracy", 500, train, train_y, dev, dev_y)

1


ValueError: ['MultinomialNB'] is not in list

### Some Resources
- https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34

In [197]:
clf = MultinomialNB()
print(clf)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
